# 基于rating2inter.ipynb生成的5-core交互图，Train/Validation/Test data splitting
- Based on generated interactions, perform data splitting


In [1]:
import os, csv
import pandas as pd

In [2]:
os.chdir('/nfs/zixuan/MMRec/Amazon_Elec_Dataset')
os.getcwd()

'/nfs/zixuan/MMRec/Amazon_Elec_Dataset'

## 直接加载现成的, Load interactions

In [3]:
rslt_file = 'elec-indexed.inter'
df = pd.read_csv(rslt_file, sep='\t')
print(f'shape: {df.shape}')
df[:4]

shape: (1689188, 5)


userID  itemID  rating   timestamp  x_label
0       0       0     5.0  1370131200        0
1       1       0     1.0  1290643200        0
2       2       0     3.0  1283990400        0
3       3       0     2.0  1290556800        0

In [4]:
import random
import numpy as np

In [5]:

df = df.sample(frac=1).reset_index(drop=True)

df.sort_values(by=['userID'], inplace=True)
df[:20]

userID  itemID  rating   timestamp  x_label
437046        0   21098     5.0  1375920000        0
1145244       0   15267     5.0  1366156800        0
1497276       0   46881     5.0  1375920000        0
1445392       0   32410     5.0  1366156800        0
1562203       0   53113     4.0  1353456000        0
313743        0   59952     5.0  1379980800        0
1682483       0       0     5.0  1370131200        0
1138367       1       0     1.0  1290643200        0
1057371       1   31019     1.0  1313107200        0
623022        1   19633     5.0  1384387200        1
261982        1   59903     3.0  1384387200        1
1275630       1   47325     3.0  1384387200        1
1277455       2   61672     3.0  1396656000        2
125091        2   42117     4.0  1362268800        0
544232        2    5224     5.0  1322179200        0
618119        2   61946     5.0  1404345600        2
968435        2   34696     5.0  1273276800        0
514602        2   26151     5.0  1273276800        0
539374        2   25074     5.0  1296345600        0
644708        2   30576     5.0  1296432000        0

In [6]:
uid_field, iid_field = 'userID', 'itemID'

uid_freq = df.groupby(uid_field)[iid_field]
u_i_dict = {}
for u, u_ls in uid_freq:
    u_i_dict[u] = list(u_ls)
u_i_dict

{0: [21098, 15267, 46881, 32410, 53113, 59952, 0],
 1: [0, 31019, 19633, 59903, 47325],
 2: [61672,
  42117,
  5224,
  61946,
  34696,
  26151,
  25074,
  30576,
  22519,
  27606,
  0,
  7302],
 3: [52587,
  43426,
  54691,
  57529,
  46150,
  50064,
  30122,
  48684,
  39896,
  23247,
  0,
  26502,
  18345,
  775,
  50933,
  27548,
  43134,
  48427,
  2746,
  42153,
  32113,
  1398],
 4: [30654, 26889, 0, 14836, 21877],
 5: [31188, 1, 32556, 54790, 33270, 9167, 20890],
 6: [58322, 1, 57374, 54572, 44370, 50885],
 7: [1, 25610, 41940, 35064, 62625, 12833, 13, 59864, 54063, 1977],
 8: [1, 30470, 41575, 56328, 57578, 40990],
 9: [32003,
  57249,
  11805,
  54096,
  40908,
  23945,
  48503,
  51111,
  56244,
  23717,
  21569,
  45501,
  13531,
  128,
  14072,
  46700,
  38977,
  42017,
  6927,
  42675,
  35300,
  32114,
  23928,
  47945,
  1164,
  30127,
  1,
  23930,
  41173,
  57515,
  37434,
  6135,
  44346,
  53023,
  32217,
  15962,
  46938,
  31569,
  12361,
  53391,
  46526,
  3200

In [7]:
new_label = []
u_ids_sorted = sorted(u_i_dict.keys())

for u in u_ids_sorted:
    items = u_i_dict[u]
    n_items = len(items)
    if n_items < 10:
        tmp_ls = [0] * (n_items - 2) + [1] + [2]
    else:
        val_test_len = int(n_items * 0.2)
        train_len = n_items - val_test_len
        val_len = val_test_len // 2
        test_len = val_test_len - val_len
        tmp_ls = [0] * train_len + [1] * val_len + [2] * test_len
    new_label.extend(tmp_ls)

new_label[:100]

[0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 2,
 2,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [8]:
df['x_label'] = new_label
df[:20]

userID  itemID  rating   timestamp  x_label
437046        0   21098     5.0  1375920000        0
1145244       0   15267     5.0  1366156800        0
1497276       0   46881     5.0  1375920000        0
1445392       0   32410     5.0  1366156800        0
1562203       0   53113     4.0  1353456000        0
313743        0   59952     5.0  1379980800        1
1682483       0       0     5.0  1370131200        2
1138367       1       0     1.0  1290643200        0
1057371       1   31019     1.0  1313107200        0
623022        1   19633     5.0  1384387200        0
261982        1   59903     3.0  1384387200        1
1275630       1   47325     3.0  1384387200        2
1277455       2   61672     3.0  1396656000        0
125091        2   42117     4.0  1362268800        0
544232        2    5224     5.0  1322179200        0
618119        2   61946     5.0  1404345600        0
968435        2   34696     5.0  1273276800        0
514602        2   26151     5.0  1273276800        0
539374        2   25074     5.0  1296345600        0
644708        2   30576     5.0  1296432000        0

In [9]:
rslt_file[:-6]

'elec-indexed'

In [10]:
new_labeled_file = rslt_file[:-6] + '-v4.inter'
df.to_csv(os.path.join('./', new_labeled_file), sep='\t', index=False)
print('done!!!')

done!!!


## Reload

In [11]:
indexed_df = pd.read_csv(new_labeled_file, sep='\t')
print(f'shape: {indexed_df.shape}')
indexed_df[:20]

shape: (1689188, 5)


userID  itemID  rating   timestamp  x_label
0        0   21098     5.0  1375920000        0
1        0   15267     5.0  1366156800        0
2        0   46881     5.0  1375920000        0
3        0   32410     5.0  1366156800        0
4        0   53113     4.0  1353456000        0
5        0   59952     5.0  1379980800        1
6        0       0     5.0  1370131200        2
7        1       0     1.0  1290643200        0
8        1   31019     1.0  1313107200        0
9        1   19633     5.0  1384387200        0
10       1   59903     3.0  1384387200        1
11       1   47325     3.0  1384387200        2
12       2   61672     3.0  1396656000        0
13       2   42117     4.0  1362268800        0
14       2    5224     5.0  1322179200        0
15       2   61946     5.0  1404345600        0
16       2   34696     5.0  1273276800        0
17       2   26151     5.0  1273276800        0
18       2   25074     5.0  1296345600        0
19       2   30576     5.0  1296432000        0

In [12]:
u_id_str, i_id_str = 'userID', 'itemID'
u_uni = indexed_df[u_id_str].unique()
c_uni = indexed_df[i_id_str].unique()

print(f'# of unique learners: {len(u_uni)}')
print(f'# of unique courses: {len(c_uni)}')

print('min/max of unique learners: {0}/{1}'.format(min(u_uni), max(u_uni)))
print('min/max of unique courses: {0}/{1}'.format(min(c_uni), max(c_uni)))


# of unique learners: 192403
# of unique courses: 63001
min/max of unique learners: 0/192402
min/max of unique courses: 0/63000
